In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
DF_VC = pd.read_csv('vehicle_claims_clean_labeled.csv')

In [5]:
DF_VC.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268255 entries, 0 to 268254
Data columns (total 26 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         268255 non-null  int64  
 1   Maker              268255 non-null  object 
 2    Genmodel          268255 non-null  object 
 3    Genmodel_ID       268255 non-null  object 
 4   Adv_year           268255 non-null  int64  
 5   Adv_month          268255 non-null  int64  
 6   Color              268255 non-null  object 
 7   Reg_year           268255 non-null  float64
 8   Bodytype           268255 non-null  object 
 9   Runned_Miles       268255 non-null  int64  
 10  Engin_size         268255 non-null  object 
 11  Gearbox            268255 non-null  object 
 12  Fuel_type          268255 non-null  object 
 13  Price              268255 non-null  float64
 14  Seat_num           268255 non-null  float64
 15  Door_num           268255 non-null  float64
 16  is

In [25]:
DF_VC

,Unnamed: 0,Maker,Genmodel,Genmodel_ID,Adv_year,Adv_month,Color,Reg_year,Bodytype,Runned_Miles,...,issue,issue_id,Adv_day,breakdown_date,repair_complexity,repair_cost,repair_hours,Label,category_anomaly,repair_date
0,0,Bentley,Arnage,10_1,2018,4,Silver,2000.0,Saloon,60000,...,Starter Motor Issue,0,19,2018-04-19,3,395.0000,9.0,0,0,2018-04-21
1,1,Bentley,Arnage,10_1,2018,6,Grey,2002.0,Saloon,44000,...,Radiator Leaking,0,15,2018-06-15,3,695.0000,6.0,0,0,2018-06-16
2,2,Bentley,Arnage,10_1,2017,11,Blue,2002.0,Saloon,55000,...,Steering Wheel Shaking,0,10,2017-11-10,3,89.9990,3.0,0,0,2017-11-10
3,3,Bentley,Arnage,10_1,2018,4,Green,2003.0,Saloon,14000,...,Electrical Issue,4,14,2018-04-14,3,224.8440,6.0,0,0,2018-04-15
4,4,Bentley,Arnage,10_1,2017,11,Grey,2003.0,Saloon,61652,...,Windscreen Crack,0,6,2017-11-06,3,75.9330,3.0,0,0,2017-11-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268250,268250,Westfield,Sport,97_1,2018,5,Yellow,2006.0,Convertible,1800,...,Starter Motor Issue,0,24,2018-05-24,2,207.5000,6.0,1,0,2018-05-25
268251,268251,Westfield,Sport,97_1,2018,5,Yellow,2006.0,Convertible,2009,...,Electrical Issue,5,3,2018-05-03,2,127.9950,6.0,1,1,2018-05-04
268252,268252,Zenos,E10,99_1,2018,3,Red,2016.0,Convertible,6,...,Windscreen Crack,0,14,2018-03-14,2,56.7700,2.0,1,0,2018-03-14
268253,268253,Zenos,E10,99_1,2018,3,Green,2016.0,Convertible,1538,...,Windscreen Crack,0,10,2018-03-10,2,60.9700,2.0,1,0,2018-03-10


In [93]:
#Step 1, cast 'Reg_year' to int
DF_VC['Reg_year'] = DF_VC['Reg_year'].apply(int)

#Step 2, cast 'Seat_num' to int
DF_VC['Seat_num'] = DF_VC['Seat_num'].apply(int)

#Step 3, cast 'Door_num' to int
DF_VC['Door_num'] = DF_VC['Door_num'].apply(int)

#Step 4, remove rows which have values outside 1 through 12 in 'Adv_month'
DF_VC = DF_VC[DF_VC['Adv_month'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])]

#Step 5, remove rows which have negative values in the 'Runned_Miles', 'Price', 'repair_hours', 'repair_cost' columns
DF_VC = DF_VC[DF_VC['Runned_Miles'] >= 0]
DF_VC = DF_VC[DF_VC['Price'] >= 0]
DF_VC = DF_VC[DF_VC['repair_hours'] >= 0]
DF_VC = DF_VC[DF_VC['repair_cost'] >= 0]

#Step 6, rename weird column names
DF_VC.rename(columns = {' Genmodel':'Genmodel', ' Genmodel_ID':'Genmodel_ID'}, inplace = True)

#Step 7, remove rows which have illogical values for 'Reg_year'
DF_VC = DF_VC[~DF_VC['Reg_year'].isin([1900, 3010])]

#Step 9, remove rows with illogical engine sizes
#DF_VC['Engin_size'] = DF_VC['Engin_size'].str[:-1]
#DF_VC['Engin_size'] = DF_VC['Engin_size'].astype(float)

DF_VC = DF_VC[(DF_VC['Engin_size'] <= 10) & (DF_VC['Engin_size'] >= 0.8)]

#Step 10, remove rows with 0 in 'Door_num'
DF_VC = DF_VC[DF_VC['Door_num'] != 0]

#Step 11,removing random outliers
DF_VC = DF_VC[DF_VC['Price'] < 3000000]
DF_VC = DF_VC[DF_VC['Seat_num'] > 1]
DF_VC = DF_VC[DF_VC['Runned_Miles'] < 6000000]
DF_VC = DF_VC[DF_VC['repair_cost'] < 140000]
DF_VC = DF_VC[DF_VC['repair_hours'] < 80]

#Step 12, removing rows for which "Reg_year" is later than "Adv_year"
DF_VC = DF_VC[DF_VC['Adv_year'] >= DF_VC['Reg_year']]

#Step 13, removing 


In [118]:

#print(sorted(DF_VC['Runned_Miles'].unique())) #outliers and ridiculously large numbers?
#print(sorted(DF_VC['Price'].unique())) #outliers and ridiculously large numbers?
#sorted(DF_VC['Door_num'].unique()) #
#print(sorted(DF_VC['repair_hours'].unique())) #decimals
#print(sorted(DF_VC['repair_cost'].unique())) #decimals
#sorted(DF_VC['Adv_day'].unique()) #


In [116]:
DF_VC

,Unnamed: 0,Maker,Genmodel,Genmodel_ID,Adv_year,Adv_month,Color,Reg_year,Bodytype,Runned_Miles,...,issue,issue_id,Adv_day,breakdown_date,repair_complexity,repair_cost,repair_hours,Label,category_anomaly,repair_date
0,0,Bentley,Arnage,10_1,2018,4,Silver,2000,Saloon,60000,...,Starter Motor Issue,0,19,2018-04-19,3,395.000000,9.0,0,0,2018-04-21
1,1,Bentley,Arnage,10_1,2018,6,Grey,2002,Saloon,44000,...,Radiator Leaking,0,15,2018-06-15,3,695.000000,6.0,0,0,2018-06-16
2,2,Bentley,Arnage,10_1,2017,11,Blue,2002,Saloon,55000,...,Steering Wheel Shaking,0,10,2017-11-10,3,89.999000,3.0,0,0,2017-11-10
3,3,Bentley,Arnage,10_1,2018,4,Green,2003,Saloon,14000,...,Electrical Issue,4,14,2018-04-14,3,224.844000,6.0,0,0,2018-04-15
4,4,Bentley,Arnage,10_1,2017,11,Grey,2003,Saloon,61652,...,Windscreen Crack,0,6,2017-11-06,3,75.933000,3.0,0,0,2017-11-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268245,268245,Volvo,V50,96_9,2018,7,Grey,2008,Estate,140000,...,Alternator Failing,0,4,2018-07-04,3,183.700000,6.0,0,0,2018-07-05
268246,268246,Volvo,V50,96_9,2018,8,Blue,2007,Estate,158000,...,Engine Issue,2,25,2018-08-25,3,1408.500000,48.0,0,0,2018-09-06
268247,268247,Volvo,V50,96_9,2018,5,Silver,2009,Estate,94000,...,Brake Pads Worn,0,22,2018-05-22,3,122.125000,6.0,0,0,2018-05-23
268248,268248,Volvo,V50,96_9,2018,5,Silver,2004,Estate,111000,...,Excessive Emissions,0,26,2018-05-26,3,62.605500,3.0,0,0,2018-05-26
